In [7]:
# !pip install imblearn

  Using cached imblearn-0.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 153kB 2.2MB/s ta 0:00:01


In [1]:
reset -fs

In [2]:
import re
from sklearn.datasets import fetch_covtype # dataset
from sklearn.model_selection import train_test_split # split dataset into training/test sets
from imblearn.under_sampling import RandomUnderSampler 
from collections import defaultdict, Counter
import pandas as pd
import numpy as np

In [3]:
# download the dataset from:
# "http://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz"
cover_type = fetch_covtype() 

In [4]:
# from the Forest_Cover_Type.ipynb data exploration we discovered there are 7 distinct cover_types
# set these covertypes as our target, y 
y = cover_type.target

print('Original dataset shape:\n {}'.format(Counter(y)))

Original dataset shape:
 Counter({2: 283301, 1: 211840, 3: 35754, 7: 20510, 6: 17367, 5: 9493, 4: 2747})


In [5]:
# Our data contains 54 features. Explored in depth within the Forest_Cover_Type.ipynb
# set this 581012 x 54 matrix as our feature matrix, X
X = cover_type.data
X.shape

(581012, 54)

### Undersample w/ Replacement

In [6]:
ros = RandomUnderSampler(random_state=42, return_indices=True, replacement=True)
X_res, y_res, idx_resampled = ros.fit_sample(X, y)
print('Resampled dataset shape: \n {}'.format(Counter(y_res)))

Resampled dataset shape: 
 Counter({1: 2747, 2: 2747, 3: 2747, 4: 2747, 5: 2747, 6: 2747, 7: 2747})


In [7]:
"Reduce to {:.2f}% of the orignal dataset.".format(X_res.shape[0]/X.shape[0] * 100)

'Reduce to 3.31% of the orignal dataset.'

In [8]:
soil_types = { 
    "2702": "Cathedral family - Rock outcrop complex, extremely stony.",
    "2703": "Vanet - Ratake families complex, very stony.", 
    "2704": "Haploborolis - Rock outcrop complex, rubbly.",
    "2705": "Ratake family - Rock outcrop complex, rubbly.",
    "2706": "Vanet family - Rock outcrop complex complex, rubbly.",
    "2717": "Vanet - Wetmore families - Rock outcrop complex, stony.",
    "3501": "Gothic family.",
    "3502": "Supervisor - Limber families complex.",
    "4201": "Troutville family, very stony.",
    "4703": "Bullwark - Catamount families - Rock outcrop complex, rubbly.",
    "4704": "Bullwark - Catamount families - Rock land complex, rubbly.",
    "4744": "Legault family - Rock land complex, stony.",
    "4758": "Catamount family - Rock land - Bullwark family complex, rubbly.",
    "5101": "Pachic Argiborolis - Aquolis complex.",
    "5151": "not_in_survey", # "unspecified in the USFS Soil and ELU Survey.",
    "6101": "Cryaquolis - Cryoborolis complex.",
    "6102": "Gateview family - Cryaquolis complex.",
    "6731": "Rogert family, very stony.",
    "7101": "Typic Cryaquolis - Borohemists complex.",
    "7102": "Typic Cryaquepts - Typic Cryaquolls complex.",
    "7103": "Typic Cryaquolls - Leighcan family, till substratum complex.",
    "7201": "Leighcan family, till substratum, extremely bouldery.",
    "7202": "Leighcan family, till substratum - Typic Cryaquolls complex.",
    "7700": "Leighcan family, extremely stony.",
    "7701": "Leighcan family, warm, extremely stony.",
    "7702": "Granile - Catamount families complex, very stony.",
    "7709": "Leighcan family, warm - Rock outcrop complex, extremely stony.",
    "7710": "Leighcan family - Rock outcrop complex, extremely stony.",
    "7745": "Como - Legault families complex, extremely stony.",
    "7746": "Como family - Rock land - Legault family complex, extremely stony.",
    "7755": "Leighcan - Catamount families complex, extremely stony.",
    "7756": "Catamount family - Rock outcrop - Leighcan family complex, extremely stony.",
    "7757": "Leighcan - Catamount families - Rock outcrop complex, extremely stony.",
    "7790": "Cryorthents - Rock land complex, extremely stony.",
    "8703": "Cryumbrepts - Rock outcrop - Cryaquepts complex.",
    "8707": "Bross family - Rock land - Cryumbrepts complex, extremely stony.",
    "8708": "Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.",
    "8771": "Leighcan - Moran families - Cryaquolls complex, extremely stony.",
    "8772": "Moran family - Cryorthents - Leighcan family complex, extremely stony.",
    "8776": "Moran family - Cryorthents - Rock land complex, extremely stony."}

In [9]:
for k, v in soil_types.items():
    fun = re.split(' - |, ', v.lower().replace(".", ""))
    colocations = [i.replace(" ", "_") for i in fun]
    soil_types[k] = " ".join(colocations)

In [10]:
soil_types

{'2702': 'cathedral_family rock_outcrop_complex extremely_stony',
 '2703': 'vanet ratake_families_complex very_stony',
 '2704': 'haploborolis rock_outcrop_complex rubbly',
 '2705': 'ratake_family rock_outcrop_complex rubbly',
 '2706': 'vanet_family rock_outcrop_complex_complex rubbly',
 '2717': 'vanet wetmore_families rock_outcrop_complex stony',
 '3501': 'gothic_family',
 '3502': 'supervisor limber_families_complex',
 '4201': 'troutville_family very_stony',
 '4703': 'bullwark catamount_families rock_outcrop_complex rubbly',
 '4704': 'bullwark catamount_families rock_land_complex rubbly',
 '4744': 'legault_family rock_land_complex stony',
 '4758': 'catamount_family rock_land bullwark_family_complex rubbly',
 '5101': 'pachic_argiborolis aquolis_complex',
 '5151': 'not_in_survey',
 '6101': 'cryaquolis cryoborolis_complex',
 '6102': 'gateview_family cryaquolis_complex',
 '6731': 'rogert_family very_stony',
 '7101': 'typic_cryaquolis borohemists_complex',
 '7102': 'typic_cryaquepts typic_c

In [11]:
# First digit:  climatic zone   
first_digit = { "1": "lower montane dry",
                "2": "lower montane",          
                "3": "montane dry",            
                "4": "montane",                
                "5": "montane dry and montane",
                "6": "montane and subalpine",
                "7": "subalpine",  
                "8": "alpine" 
              }  

# Second digit:  geologic zones
second_digit = {"1": "alluvium",
                "2": "glacial",
                "3": "shale",
                "4": "sandstone",
                "5": "mixed sedimentary",
                "6": "not_in_survey", #"unspecified in the USFS ELU Survey"
                "7": "igneous and metamorphic",
                "8": "volcanic"
               }

# The third and fourth ELU digits are unique to the mapping unit and 
# have no special meaning to the climatic or geologic zones.

In [12]:
for k, v in first_digit.items():
    first_digit[k] = v.replace(" ", "_")

In [13]:
first_digit

{'1': 'lower_montane_dry',
 '2': 'lower_montane',
 '3': 'montane_dry',
 '4': 'montane',
 '5': 'montane_dry_and_montane',
 '6': 'montane_and_subalpine',
 '7': 'subalpine',
 '8': 'alpine'}

In [14]:
for k, v in second_digit.items():
    second_digit[k] = v.replace(" ", "_")

In [15]:
second_digit

{'1': 'alluvium',
 '2': 'glacial',
 '3': 'shale',
 '4': 'sandstone',
 '5': 'mixed_sedimentary',
 '6': 'not_in_survey',
 '7': 'igneous_and_metamorphic',
 '8': 'volcanic'}

In [16]:
soil_types_extend = defaultdict(str, soil_types)

for k in soil_types_extend.keys():
    climatic = "climatic_zone_" + first_digit.get(k[0])
    geologic = "geologic_zone_" + second_digit.get(k[1])
    soil_types_extend[k] += " " + climatic + " " + geologic

In [17]:
soil_types_extend

defaultdict(str,
            {'2702': 'cathedral_family rock_outcrop_complex extremely_stony climatic_zone_lower_montane geologic_zone_igneous_and_metamorphic',
             '2703': 'vanet ratake_families_complex very_stony climatic_zone_lower_montane geologic_zone_igneous_and_metamorphic',
             '2704': 'haploborolis rock_outcrop_complex rubbly climatic_zone_lower_montane geologic_zone_igneous_and_metamorphic',
             '2705': 'ratake_family rock_outcrop_complex rubbly climatic_zone_lower_montane geologic_zone_igneous_and_metamorphic',
             '2706': 'vanet_family rock_outcrop_complex_complex rubbly climatic_zone_lower_montane geologic_zone_igneous_and_metamorphic',
             '2717': 'vanet wetmore_families rock_outcrop_complex stony climatic_zone_lower_montane geologic_zone_igneous_and_metamorphic',
             '3501': 'gothic_family climatic_zone_montane_dry geologic_zone_mixed_sedimentary',
             '3502': 'supervisor limber_families_complex climatic_zone

In [18]:
# The wilderness areas are
wilderness_areas =   {'Wilderness_Area1': "Rawah Wilderness Area", 
                      'Wilderness_Area2': "Neota Wilderness Area",
                      'Wilderness_Area3': "Comanche Peak Wilderness Area",
                      'Wilderness_Area4': "Cache la Poudre Wilderness Area"}

In [19]:
for k, v in wilderness_areas.items():
    wilderness_areas[k] = v.lower().replace(" ", "_")

In [20]:
wilderness_areas

{'Wilderness_Area1': 'rawah_wilderness_area',
 'Wilderness_Area2': 'neota_wilderness_area',
 'Wilderness_Area3': 'comanche_peak_wilderness_area',
 'Wilderness_Area4': 'cache_la_poudre_wilderness_area'}

In [21]:
soil_cols = list(soil_types_extend.values())
wilderness_cols = list(wilderness_areas.values())

In [22]:
# cols = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
#        'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
#        'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
#        'Horizontal_Distance_To_Fire_Points'] 
# cols += wilderness_cols + soil_cols

In [23]:
wild_test = pd.DataFrame(X_res[:, 10:14], columns=wilderness_cols).head()
wild_test

,rawah_wilderness_area,neota_wilderness_area,comanche_peak_wilderness_area,cache_la_poudre_wilderness_area
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [24]:
pd.Series(wild_test.idxmax(axis=1))

0    comanche_peak_wilderness_area
1    comanche_peak_wilderness_area
2    comanche_peak_wilderness_area
3    comanche_peak_wilderness_area
4    comanche_peak_wilderness_area
dtype: object

In [25]:
soil_test = pd.DataFrame(X_res[:, 14:], columns=soil_cols).head()
soil_test

,cathedral_family rock_outcrop_complex extremely_stony climatic_zone_lower_montane geologic_zone_igneous_and_metamorphic,vanet ratake_families_complex very_stony climatic_zone_lower_montane geologic_zone_igneous_and_metamorphic,haploborolis rock_outcrop_complex rubbly climatic_zone_lower_montane geologic_zone_igneous_and_metamorphic,ratake_family rock_outcrop_complex rubbly climatic_zone_lower_montane geologic_zone_igneous_and_metamorphic,vanet_family rock_outcrop_complex_complex rubbly climatic_zone_lower_montane geologic_zone_igneous_and_metamorphic,vanet wetmore_families rock_outcrop_complex stony climatic_zone_lower_montane geologic_zone_igneous_and_metamorphic,gothic_family climatic_zone_montane_dry geologic_zone_mixed_sedimentary,supervisor limber_families_complex climatic_zone_montane_dry geologic_zone_mixed_sedimentary,troutville_family very_stony climatic_zone_montane geologic_zone_glacial,bullwark catamount_families rock_outcrop_complex rubbly climatic_zone_montane geologic_zone_igneous_and_metamorphic,...,leighcan catamount_families_complex extremely_stony climatic_zone_subalpine geologic_zone_igneous_and_metamorphic,catamount_family rock_outcrop leighcan_family_complex extremely_stony climatic_zone_subalpine geologic_zone_igneous_and_metamorphic,leighcan catamount_families rock_outcrop_complex extremely_stony climatic_zone_subalpine geologic_zone_igneous_and_metamorphic,cryorthents rock_land_complex extremely_stony climatic_zone_subalpine geologic_zone_igneous_and_metamorphic,cryumbrepts rock_outcrop cryaquepts_complex climatic_zone_alpine geologic_zone_igneous_and_metamorphic,bross_family rock_land cryumbrepts_complex extremely_stony climatic_zone_alpine geologic_zone_igneous_and_metamorphic,rock_outcrop cryumbrepts cryorthents_complex extremely_stony climatic_zone_alpine geologic_zone_igneous_and_metamorphic,leighcan moran_families cryaquolls_complex extremely_stony climatic_zone_alpine geologic_zone_igneous_and_metamorphic,moran_family cryorthents leighcan_family_complex extremely_stony climatic_zone_alpine geologic_zone_igneous_and_metamorphic,moran_family cryorthents rock_land_complex extremely_stony climatic_zone_alpine geologic_zone_igneous_and_metamorphic
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
wild_test.idxmax(axis=1) + ". " + soil_test.idxmax(axis=1)

0    comanche_peak_wilderness_area. catamount_famil...
1    comanche_peak_wilderness_area. moran_family cr...
2    comanche_peak_wilderness_area. leighcan catamo...
3    comanche_peak_wilderness_area. catamount_famil...
4    comanche_peak_wilderness_area. leighcan_family...
dtype: object

In [27]:
wild_df = pd.DataFrame(X_res[:, 10:14], columns=wilderness_cols)

soil_df = pd.DataFrame(X_res[:, 14:], columns=soil_cols)

X_wild_soil = wild_df.idxmax(axis=1) + " " + soil_df.idxmax(axis=1)
X_wild_soil.head(10)

0    comanche_peak_wilderness_area catamount_family...
1    comanche_peak_wilderness_area moran_family cry...
2    comanche_peak_wilderness_area leighcan catamou...
3    comanche_peak_wilderness_area catamount_family...
4    comanche_peak_wilderness_area leighcan_family ...
5    comanche_peak_wilderness_area catamount_family...
6    neota_wilderness_area catamount_family rock_ou...
7    rawah_wilderness_area como legault_families_co...
8    rawah_wilderness_area leighcan_family till_sub...
9    comanche_peak_wilderness_area catamount_family...
dtype: object

In [28]:
X_wild_soil[0]

'comanche_peak_wilderness_area catamount_family rock_outcrop leighcan_family_complex extremely_stony climatic_zone_subalpine geologic_zone_igneous_and_metamorphic'

In [58]:
targets_res = pd.Series(y_res) #targets from random undersampler 
X_wild_soil_w_targets = pd.DataFrame({"wild_soil": X_wild_soil, "soil_type":targets_res})
X_wild_soil_w_targets.head()

,soil_type,wild_soil
0,1,comanche_peak_wilderness_area catamount_family...
1,1,comanche_peak_wilderness_area moran_family cry...
2,1,comanche_peak_wilderness_area leighcan catamou...
3,1,comanche_peak_wilderness_area catamount_family...
4,1,comanche_peak_wilderness_area leighcan_family ...


In [127]:
test = Counter()

a = query[0]
b = list(a.split(" "))
test.update(b)
test

Counter({'Catamount_family,': 1,
         'Comanche_Peak_Wilderness_Area.': 1,
         'Leighcan_family_complex,': 1,
         'Rock_outcrop,': 1,
         'climatic_zone_subalpine,': 1,
         'extremely_stony,': 1,
         'geologic_zone_igneous_and_metamorphic': 1})

In [60]:
i = 1
query = X_wild_soil_w_targets.query("soil_type == {}".format(i))["wild_soil"]
test = Counter()
for row in query:
    words = list(row.split(" "))
    test.update(words)
test.most_common(10)

[('climatic_zone_subalpine', 2352),
 ('geologic_zone_igneous_and_metamorphic', 1831),
 ('extremely_stony', 1721),
 ('rawah_wilderness_area', 1357),
 ('comanche_peak_wilderness_area', 1140),
 ('leighcan_family', 985),
 ('geologic_zone_glacial', 824),
 ('till_substratum', 821),
 ('como', 535),
 ('legault_families_complex', 535)]

In [65]:
[i[0] for i in test.most_common(10)]

['climatic_zone_subalpine',
 'geologic_zone_igneous_and_metamorphic',
 'extremely_stony',
 'rawah_wilderness_area',
 'comanche_peak_wilderness_area',
 'leighcan_family',
 'geologic_zone_glacial',
 'till_substratum',
 'como',
 'legault_families_complex']

In [66]:
soil_CountVect = dict()
for i in range(1, 8): 
    print("Soil Type {}:".format(i))
    query = X_wild_soil_w_targets.query("soil_type == {}".format(i))["wild_soil"]
    temp_counter = Counter()
    for row in query:
        words = list(row.split(" "))
        temp_counter.update(words)
    top_10 = [i[0] for i in temp_counter.most_common(10)]
    print("Unique words in Soil Type {} vocabulary: {}".format(i, len(temp_counter)))
    print("Top 10 words in Soil Type {}: {} \n".format(i, top_10))        
    soil_CountVect["Soil Type {}:".format(i)] = ", ".join(top_10)

Soil Type 1:
Unique words in Soil Type 1 vocabulary: 57
Top 10 words in Soil Type 1: ['climatic_zone_subalpine', 'geologic_zone_igneous_and_metamorphic', 'extremely_stony', 'rawah_wilderness_area', 'comanche_peak_wilderness_area', 'leighcan_family', 'geologic_zone_glacial', 'till_substratum', 'como', 'legault_families_complex'] 

Soil Type 2:
Unique words in Soil Type 2 vocabulary: 61
Top 10 words in Soil Type 2: ['geologic_zone_igneous_and_metamorphic', 'climatic_zone_subalpine', 'extremely_stony', 'rawah_wilderness_area', 'comanche_peak_wilderness_area', 'como', 'legault_families_complex', 'climatic_zone_montane', 'catamount_families', 'rock_outcrop_complex'] 

Soil Type 3:
Unique words in Soil Type 3 vocabulary: 36
Top 10 words in Soil Type 3: ['geologic_zone_igneous_and_metamorphic', 'rock_outcrop_complex', 'rubbly', 'climatic_zone_lower_montane', 'cache_la_poudre_wilderness_area', 'comanche_peak_wilderness_area', 'climatic_zone_montane', 'bullwark', 'catamount_families', 'vanet'] 

In [67]:
from sklearn.feature_extraction.text import CountVectorizer

In [71]:
# CountVectorizer for entire dataset
min_df = 1 # default making a point to keep all features if min_df=2 then only token removed is 'not_in_survey'
max_df = 0.95 # unless they appear in all docs 
max_features = 100
vectorizer = CountVectorizer(max_features=max_features, max_df=max_df, min_df=min_df)

vectorized = vectorizer.fit_transform(X_wild_soil)
vectorized.shape, vectorizer.stop_words_

((19229, 75), set())

In [73]:
from sklearn.decomposition import NMF

In [74]:
model = NMF(init="nndsvd",
            n_components=7,
            max_iter=200)

In [75]:
W = model.fit_transform(vectorized)
H = model.components_

In [76]:
W.shape, H.shape

((19229, 7), (7, 75))

In [77]:
terms = [""] * len(vectorizer.vocabulary_)
for term in vectorizer.vocabulary_.keys():
    terms[vectorizer.vocabulary_[term]] = term

In [79]:
for topic_index in range(H.shape[0]):
    top_indicies = np.argsort(H[topic_index, :])[::-1][0:10]
    term_ranking = [terms[i] for i in top_indicies]
    print("Soil Topic {}: {} \n".format(topic_index+1, ", ".join(term_ranking)))

Soil Topic 1: catamount_families, bullwark, climatic_zone_montane, rubbly, rock_outcrop_complex, geologic_zone_igneous_and_metamorphic, cache_la_poudre_wilderness_area, rock_land_complex, comanche_peak_wilderness_area, legault_family 

Soil Topic 2: rawah_wilderness_area, geologic_zone_igneous_and_metamorphic, extremely_stony, climatic_zone_subalpine, legault_families_complex, como, legault_family_complex, como_family, rock_land, legault_family 

Soil Topic 3: climatic_zone_lower_montane, cache_la_poudre_wilderness_area, geologic_zone_igneous_and_metamorphic, rock_outcrop_complex, rubbly, vanet, haploborolis, stony, wetmore_families, ratake_family 

Soil Topic 4: leighcan_family, geologic_zone_glacial, till_substratum, climatic_zone_subalpine, typic_cryaquolls_complex, rawah_wilderness_area, extremely_bouldery, comanche_peak_wilderness_area, neota_wilderness_area, geologic_zone_alluvium 

Soil Topic 5: extremely_stony, comanche_peak_wilderness_area, climatic_zone_subalpine, geologic_zo

In [81]:
for k, v in soil_CountVect.items():
    v_str = v
    print(k, v_str)
    print("")

Soil Type 1: climatic_zone_subalpine, geologic_zone_igneous_and_metamorphic, extremely_stony, rawah_wilderness_area, comanche_peak_wilderness_area, leighcan_family, geologic_zone_glacial, till_substratum, como, legault_families_complex

Soil Type 2: geologic_zone_igneous_and_metamorphic, climatic_zone_subalpine, extremely_stony, rawah_wilderness_area, comanche_peak_wilderness_area, como, legault_families_complex, climatic_zone_montane, catamount_families, rock_outcrop_complex

Soil Type 3: geologic_zone_igneous_and_metamorphic, rock_outcrop_complex, rubbly, climatic_zone_lower_montane, cache_la_poudre_wilderness_area, comanche_peak_wilderness_area, climatic_zone_montane, bullwark, catamount_families, vanet

Soil Type 4: cache_la_poudre_wilderness_area, geologic_zone_igneous_and_metamorphic, rock_outcrop_complex, climatic_zone_lower_montane, rubbly, haploborolis, geologic_zone_alluvium, climatic_zone_montane_and_subalpine, gateview_family, cryaquolis_complex

Soil Type 5: geologic_zone_

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [86]:
# Tfidf on entire dataset (balanced)
min_df = 1 # default making a point to keep all features 
max_df = 0.95 # unless they appear in all docs 
max_features = 100 # not a problem here... soil + wilderness has a max of 70 feats

tfidf_vec = TfidfVectorizer(max_features=max_features, max_df=max_df, min_df=min_df)

tfidf_vecD = tfidf_vec.fit_transform(X_wild_soil)
tfidf_vecD.shape, tfidf_vec.stop_words_

((19229, 75), set())

In [87]:
terms = [""] * len(tfidf_vec.vocabulary_)
for term in tfidf_vec.vocabulary_.keys():
    terms[tfidf_vec.vocabulary_[term]] = term

In [88]:
model_tfidf = NMF(init="nndsvd",
                n_components=7,
                max_iter=200)

W_tfidf = model_tfidf.fit_transform(tfidf_vecD)
H_tfidf = model_tfidf.components_

W_tfidf.shape, H_tfidf.shape

((19229, 7), (7, 75))

In [90]:
for topic_index in range(H_tfidf.shape[0]):
    top_indices = np.argsort(H_tfidf[topic_index,:])[::-1][0:10]
    term_ranking = [terms[i] for i in top_indices]
    print("Soil Type {}: {}\n".format(topic_index+1, ", ".join(term_ranking)))

Soil Type 1: bullwark, climatic_zone_montane, catamount_families, rubbly, rock_outcrop_complex, cache_la_poudre_wilderness_area, geologic_zone_igneous_and_metamorphic, comanche_peak_wilderness_area, rock_land_complex, bullwark_family_complex

Soil Type 2: legault_families_complex, como, rawah_wilderness_area, climatic_zone_subalpine, extremely_stony, geologic_zone_igneous_and_metamorphic, cathedral_family, legault_family, leighcan_family, stony

Soil Type 3: climatic_zone_lower_montane, rock_outcrop_complex, cache_la_poudre_wilderness_area, haploborolis, rubbly, geologic_zone_igneous_and_metamorphic, ratake_family, vanet, wetmore_families, stony

Soil Type 4: cryorthents, moran_family, climatic_zone_alpine, leighcan_family_complex, extremely_stony, rock_land_complex, comanche_peak_wilderness_area, geologic_zone_igneous_and_metamorphic, rock_outcrop, neota_wilderness_area

Soil Type 5: leighcan_family, till_substratum, geologic_zone_glacial, typic_cryaquolls_complex, climatic_zone_subal

In [91]:
for k, v in soil_CountVect.items():
    v_str = v
    print(k, v_str)
    print("")

Soil Type 1: climatic_zone_subalpine, geologic_zone_igneous_and_metamorphic, extremely_stony, rawah_wilderness_area, comanche_peak_wilderness_area, leighcan_family, geologic_zone_glacial, till_substratum, como, legault_families_complex

Soil Type 2: geologic_zone_igneous_and_metamorphic, climatic_zone_subalpine, extremely_stony, rawah_wilderness_area, comanche_peak_wilderness_area, como, legault_families_complex, climatic_zone_montane, catamount_families, rock_outcrop_complex

Soil Type 3: geologic_zone_igneous_and_metamorphic, rock_outcrop_complex, rubbly, climatic_zone_lower_montane, cache_la_poudre_wilderness_area, comanche_peak_wilderness_area, climatic_zone_montane, bullwark, catamount_families, vanet

Soil Type 4: cache_la_poudre_wilderness_area, geologic_zone_igneous_and_metamorphic, rock_outcrop_complex, climatic_zone_lower_montane, rubbly, haploborolis, geologic_zone_alluvium, climatic_zone_montane_and_subalpine, gateview_family, cryaquolis_complex

Soil Type 5: geologic_zone_

In [92]:
from sklearn.decomposition.online_lda import LatentDirichletAllocation

In [94]:
lda = LatentDirichletAllocation(n_components=7,
                                max_iter=5,
                                learning_method='online',
                                learning_offset=50,
                                random_state=42)

In [219]:
lda_norm = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]

In [222]:
lda_norm[4]

array([  3.28146925e-05,   3.28232421e-05,   3.28147258e-05,
         3.28147007e-05,   3.28146729e-05,   3.28148334e-05,
         3.28154929e-05,   3.28158210e-05,   3.28184314e-05,
         3.28146866e-05,   3.28239795e-05,   3.28146603e-05,
         1.19141712e-02,   3.28172115e-05,   3.28176594e-05,
         3.28146947e-05,   1.56319088e-01,   3.28320262e-05,
         1.54291548e-01,   3.29442356e-05,   3.28228843e-05,
         3.28183963e-05,   3.28146586e-05,   3.28226802e-05,
         3.28184010e-05,   3.28146432e-05,   3.28177959e-05,
         3.28217504e-05,   3.28147372e-05,   3.28145898e-05,
         1.55615405e-01,   3.28146475e-05,   3.28174173e-05,
         3.28145860e-05,   8.18844690e-02,   3.28176989e-05,
         3.28201859e-05,   3.28145915e-05,   3.28146383e-05,
         1.54291548e-01,   2.45012776e-02,   3.29445064e-05,
         3.28199412e-05,   3.28276167e-05,   3.28183663e-05,
         3.28182070e-05,   3.28227094e-05,   3.28146391e-05,
         3.28164815e-05,

In [95]:
lda.fit(tfidf_vecD)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=7, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [189]:
b[0].argsort()[:-11:-1]

array([ 8, 56, 44, 17, 16, 30, 34, 66, 32, 69])

In [190]:
tf_feature_names = tfidf_vec.get_feature_names()
lda_topics = dict()
for topic_idx, topic in enumerate(lda.components_):
    lda_topics["Topic {}:".format(topic_idx+1)] = [i for i in topic.argsort()[:-11:-1]] # keep track of indicies 
    print("Topic {}:".format(topic_idx+1), ", ".join([tf_feature_names[i] for i in topic.argsort()[:-11:-1]]))
    print("")

Topic 1: catamount_family, rock_outcrop, leighcan_family_complex, comanche_peak_wilderness_area, climatic_zone_subalpine, extremely_stony, geologic_zone_igneous_and_metamorphic, typic_cryaquepts, geologic_zone_alluvium, typic_cryaquolls_complex

Topic 2: climatic_zone_montane, bullwark, rubbly, cache_la_poudre_wilderness_area, rock_land, geologic_zone_igneous_and_metamorphic, como_family, legault_family_complex, climatic_zone_montane_and_subalpine, bullwark_family_complex

Topic 3: cryorthents, moran_family, climatic_zone_alpine, rock_land_complex, extremely_stony, leighcan_family_complex, comanche_peak_wilderness_area, geologic_zone_igneous_and_metamorphic, neota_wilderness_area, catamount_families_complex

Topic 4: catamount_families, rock_outcrop_complex, leighcan, comanche_peak_wilderness_area, cryaquolls_complex, moran_families, geologic_zone_igneous_and_metamorphic, extremely_stony, climatic_zone_alpine, climatic_zone_subalpine

Topic 5: rawah_wilderness_area, climatic_zone_subal

In [199]:
for k, v in soil_CountVect.items():
    print(k, v)
    print("")

Soil Type 1: climatic_zone_subalpine, geologic_zone_igneous_and_metamorphic, extremely_stony, rawah_wilderness_area, comanche_peak_wilderness_area, leighcan_family, geologic_zone_glacial, till_substratum, como, legault_families_complex

Soil Type 2: geologic_zone_igneous_and_metamorphic, climatic_zone_subalpine, extremely_stony, rawah_wilderness_area, comanche_peak_wilderness_area, como, legault_families_complex, climatic_zone_montane, catamount_families, rock_outcrop_complex

Soil Type 3: geologic_zone_igneous_and_metamorphic, rock_outcrop_complex, rubbly, climatic_zone_lower_montane, cache_la_poudre_wilderness_area, comanche_peak_wilderness_area, climatic_zone_montane, bullwark, catamount_families, vanet

Soil Type 4: cache_la_poudre_wilderness_area, geologic_zone_igneous_and_metamorphic, rock_outcrop_complex, climatic_zone_lower_montane, rubbly, haploborolis, geologic_zone_alluvium, climatic_zone_montane_and_subalpine, gateview_family, cryaquolis_complex

Soil Type 5: geologic_zone_

In [191]:
lda_topics

{'Topic 1:': [8, 56, 44, 17, 16, 30, 34, 66, 32, 69],
 'Topic 2:': [12, 3, 60, 5, 54, 34, 19, 41, 13, 4],
 'Topic 3:': [25, 47, 10, 55, 30, 44, 17, 34, 48, 7],
 'Topic 4:': [6, 57, 42, 17, 23, 46, 34, 30, 10, 16],
 'Topic 5:': [53, 16, 30, 39, 18, 34, 40, 61, 55, 12],
 'Topic 6:': [11, 57, 5, 60, 34, 38, 70, 52, 72, 74],
 'Topic 7:': [43, 33, 63, 69, 16, 17, 53, 29, 48, 68]}

In [207]:
soil_CountVect["Soil Type 1:"].split(", ")

['climatic_zone_subalpine',
 'geologic_zone_igneous_and_metamorphic',
 'extremely_stony',
 'rawah_wilderness_area',
 'comanche_peak_wilderness_area',
 'leighcan_family',
 'geologic_zone_glacial',
 'till_substratum',
 'como',
 'legault_families_complex']

In [210]:
test_true = list()
for y_true, y_pred in zip(soil_CountVect["Soil Type 1:"].split(", "), lda_topics["Topic 5:"]):
    test_true.append(tf_feature_names.index(y_true))
    print(tf_feature_names.index(y_true), y_pred)

16 53
34 16
30 30
53 39
17 18
43 34
33 40
63 61
18 55
39 12


In [101]:
from sklearn.metrics import log_loss

In [212]:
# True soil type 1
print(test_true)
# predicted "soil type 1" from lda --> "topic 5"
test_pred = np.diagflat(lda_topics["Topic 5:"])
print(test_pred)

log_loss(test_true, test_pred)

[16, 34, 30, 53, 17, 43, 33, 63, 18, 39]
[[53  0  0  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  0  0  0  0  0]
 [ 0  0 30  0  0  0  0  0  0  0]
 [ 0  0  0 39  0  0  0  0  0  0]
 [ 0  0  0  0 18  0  0  0  0  0]
 [ 0  0  0  0  0 34  0  0  0  0]
 [ 0  0  0  0  0  0 40  0  0  0]
 [ 0  0  0  0  0  0  0 61  0  0]
 [ 0  0  0  0  0  0  0  0 55  0]
 [ 0  0  0  0  0  0  0  0  0 12]]


31.084898755419623

In [264]:
log_loss(test_true, test_pred, labels=test_true)

31.084898755419623

In [241]:
test_true2 = list()
for y_true in test.most_common():
    test_true2.append(tf_feature_names.index(y_true[0]))
test_true2

[16,
 34,
 30,
 53,
 17,
 43,
 33,
 63,
 18,
 39,
 69,
 42,
 44,
 29,
 8,
 10,
 56,
 6,
 57,
 48,
 47,
 25,
 7,
 55,
 54,
 46,
 23,
 12,
 32,
 19,
 41,
 66,
 60,
 40,
 61,
 4,
 67,
 1,
 3,
 13,
 72,
 27,
 20,
 73,
 31,
 22,
 37,
 68,
 64,
 21,
 24,
 65,
 59,
 52,
 11,
 2,
 28]

In [257]:
wtf_mate = np.array((list(lda_norm[4][test_true2]) * len(test_true2))).reshape(len(test_true2),len(test_true2))

In [258]:
log_loss(test_true2, wtf_mate, labels=test_true2)

8.9982320008088728

In [261]:
log_loss(test_true2, np.diagflat(lda_norm[4][test_true2]))

25.539953534362336

In [224]:
from sklearn.ensemble import RandomForestClassifier
# fitting our training data with scikit learn's RandomForestClassifier
# Choosing this algorithm over others due to our feature space. 
# 44 of our features are binary, whether or not the tree is in one of 4 wilderness areas
# and whether or not the tree is found in one of 40 soil types
# In the Forest_Cover_Type.ipynb, the goal was to correctly classify only one tree type (7)
# Through data exploration, and the logit function it was found that nearly all of the 
# features were statistically significant. 
clf = RandomForestClassifier()
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [229]:
a = clf.predict_proba(X)[:5]
a

array([[ 0. ,  0.1,  0. ,  0. ,  0.9,  0. ,  0. ],
       [ 0. ,  0.1,  0. ,  0. ,  0.9,  0. ,  0. ],
       [ 0.1,  0.9,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ]])

In [230]:
y[:5]

array([5, 5, 2, 2, 5], dtype=int32)

In [233]:
log_loss(y[:5], a, labels=np.arange(1,8))

0.063216309394701575

In [17]:
# using only 10 trees, we are able to predict with a very high accuracy
clf.score(X_test, y_test)

0.9441155209803449

In [18]:
clf.classes_

array([1, 2, 3, 4, 5, 6, 7], dtype=int32)

In [19]:
clf.estimators_[0]

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=95787995, splitter='best')

In [32]:
def srted_index():
    idx_lst = []
    for i, feat in enumerate(clf.feature_importances_):
        idx_lst.append([feat, i])
    return sorted(idx_lst, reverse=True)

In [33]:
srted_index()

[[0.24660715000309424, 0],
 [0.11880706413405626, 5],
 [0.11152332641382312, 9],
 [0.059908103854401686, 3],
 [0.05645230632134722, 4],
 [0.047170463008234624, 1],
 [0.041989991641566075, 7],
 [0.041393662003483399, 6],
 [0.040573691144154225, 8],
 [0.03374382486116325, 2],
 [0.022282472125462558, 13],
 [0.016397530318239713, 35],
 [0.013449267507324187, 17],
 [0.013250193937924695, 23],
 [0.011661033568579889, 12],
 [0.010540464318108169, 15],
 [0.010170187339738216, 36],
 [0.010121195893610394, 25],
 [0.0096949731577236822, 10],
 [0.0091872477096017341, 52],
 [0.0089744993140728011, 51],
 [0.006788750888043503, 11],
 [0.0055391919733450423, 45],
 [0.0054293017962487528, 53],
 [0.0053588437574285715, 19],
 [0.0049212520631274469, 42],
 [0.0047951413589761554, 46],
 [0.0041894587820109786, 26],
 [0.0040481499729057464, 37],
 [0.0040246374942260969, 44],
 [0.0031794358396381893, 43],
 [0.0022974648204542028, 16],
 [0.0020622543592601233, 24],
 [0.0019328281663092354, 48],
 [0.0018229078

In [17]:
clf100 = RandomForestClassifier(n_estimators=100, n_jobs=-1, verbose=1)
clf100.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   48.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.8min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=1, warm_start=False)

In [18]:
# No surprise, more trees, more accurate 
clf100.score(X_test, y_test)

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.8s finished


0.95633541014078693

In [34]:
param_grid = dict(n_estimators=[25, 50, 100, 200],
                  max_depth=np.arange(2,10),
                  max_features=np.arange(7, 14),
                  max_leaf_nodes=np.arange(2,8)
                 )

grid = RandomizedSearchCV(estimator=RandomForestClassifier(),
                         n_iter=10,
                         param_distributions=param_grid,
                         cv=5,
                         n_jobs=-1,
                         verbose=2)

grid = grid.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=25, max_leaf_nodes=7, max_features=7, max_depth=6 ..
[CV] n_estimators=25, max_leaf_nodes=7, max_features=7, max_depth=6 ..
[CV] n_estimators=25, max_leaf_nodes=7, max_features=7, max_depth=6 ..
[CV] n_estimators=25, max_leaf_nodes=7, max_features=7, max_depth=6 ..
[CV]  n_estimators=25, max_leaf_nodes=7, max_features=7, max_depth=6, total=  25.4s
[CV] n_estimators=25, max_leaf_nodes=7, max_features=7, max_depth=6 ..
[CV]  n_estimators=25, max_leaf_nodes=7, max_features=7, max_depth=6, total=  27.5s
[CV] n_estimators=200, max_leaf_nodes=6, max_features=9, max_depth=4 .
[CV]  n_estimators=25, max_leaf_nodes=7, max_features=7, max_depth=6, total=  27.2s
[CV] n_estimators=200, max_leaf_nodes=6, max_features=9, max_depth=4 .
[CV]  n_estimators=25, max_leaf_nodes=7, max_features=7, max_depth=6, total=  27.3s
[CV] n_estimators=200, max_leaf_nodes=6, max_features=9, max_depth=4 .
[CV]  n_estimators=25, max_leaf_nod

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 15.3min


[CV]  n_estimators=50, max_leaf_nodes=4, max_features=7, max_depth=7, total=  36.4s
[CV] n_estimators=200, max_leaf_nodes=7, max_features=12, max_depth=8 
[CV]  n_estimators=100, max_leaf_nodes=7, max_features=13, max_depth=6, total= 2.8min
[CV] n_estimators=200, max_leaf_nodes=7, max_features=12, max_depth=8 
[CV]  n_estimators=200, max_leaf_nodes=7, max_features=12, max_depth=8, total= 5.2min
[CV] n_estimators=200, max_leaf_nodes=7, max_features=12, max_depth=8 
[CV]  n_estimators=200, max_leaf_nodes=7, max_features=12, max_depth=8, total= 5.4min
[CV] n_estimators=50, max_leaf_nodes=2, max_features=9, max_depth=6 ..
[CV]  n_estimators=200, max_leaf_nodes=7, max_features=12, max_depth=8, total= 5.5min
[CV] n_estimators=50, max_leaf_nodes=2, max_features=9, max_depth=6 ..
[CV]  n_estimators=50, max_leaf_nodes=2, max_features=9, max_depth=6, total=  32.6s
[CV] n_estimators=50, max_leaf_nodes=2, max_features=9, max_depth=6 ..
[CV]  n_estimators=50, max_leaf_nodes=2, max_features=9, max_d

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 24.8min finished


In [35]:
grid.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features=13, max_leaf_nodes=7,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [36]:
grid.best_params_

{'max_depth': 6, 'max_features': 13, 'max_leaf_nodes': 7, 'n_estimators': 100}

In [37]:
grid.best_score_

0.67040599720793248

In [41]:
param_grid = dict(n_estimators=[25, 50, 100, 200],
                  max_features=np.arange(7, 14)
                 )

grid_stump = RandomizedSearchCV(estimator=RandomForestClassifier(max_depth=1),
                         n_iter=10,
                         param_distributions=param_grid,
                         cv=5,
                         n_jobs=-1,
                         verbose=2)

grid_stump = grid_stump.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=200, max_features=10 ...............................
[CV] n_estimators=200, max_features=10 ...............................
[CV] n_estimators=200, max_features=10 ...............................
[CV] n_estimators=200, max_features=10 ...............................
[CV] ................ n_estimators=200, max_features=10, total= 1.2min
[CV] n_estimators=200, max_features=10 ...............................
[CV] ................ n_estimators=200, max_features=10, total= 1.2min
[CV] n_estimators=50, max_features=13 ................................
[CV] ................ n_estimators=200, max_features=10, total= 1.2min
[CV] ................ n_estimators=200, max_features=10, total= 1.2min
[CV] n_estimators=50, max_features=13 ................................
[CV] n_estimators=50, max_features=13 ................................
[CV] ................. n_estimators=50, max_features=13, total=  21.1s
[CV] n_estimator

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 17.0min


[CV] ................. n_estimators=25, max_features=13, total=  16.0s
[CV] n_estimators=25, max_features=13 ................................
[CV] ................. n_estimators=25, max_features=13, total=  15.6s
[CV] n_estimators=25, max_features=13 ................................
[CV] ................. n_estimators=25, max_features=13, total=  16.2s
[CV] n_estimators=25, max_features=13 ................................
[CV] ................. n_estimators=200, max_features=9, total= 2.0min
[CV] n_estimators=25, max_features=12 ................................
[CV] ................. n_estimators=25, max_features=13, total=  18.5s
[CV] n_estimators=25, max_features=12 ................................
[CV] ................. n_estimators=25, max_features=13, total=  16.4s
[CV] n_estimators=25, max_features=12 ................................
[CV] ................. n_estimators=25, max_features=12, total=  13.2s
[CV] n_estimators=25, max_features=12 ................................
[CV] .

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 19.0min finished


In [42]:
grid_stump.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=1, max_features=13, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [43]:
grid_stump.best_params_

{'max_features': 13, 'n_estimators': 100}

In [44]:
grid_stump.best_score_

0.59520758830391463

In [45]:
from collections import Counter
import numpy as np

In [46]:
class TreeNode(object):
    '''
    A node class for a decision tree.
    '''
    def __init__(self):
        self.column = None  # (int)    index of feature to split on
        self.value = None  # value of the feature to split on
        self.categorical = True  # (bool) whether or not node is split on
                                 # categorial feature
        self.name = None    # (string) name of feature (or name of class in the
                            #          case of a list)
        self.left = None    # (TreeNode) left child
        self.right = None   # (TreeNode) right child
        self.leaf = False   # (bool)   true if node is a leaf, false otherwise
        self.classes = Counter()  # (Counter) only necessary for leaf node:
                                  #           key is class name and value is
                                  #           count of the count of data points
                                  #           that terminate at this leaf

    def predict_one(self, x):
        '''
        INPUT:
            - x: 1d numpy array (single data point)
        OUTPUT:
            - y: predicted label
        Return the predicted label for a single data point.
        '''
        if self.leaf:
            return self.name
        col_value = x[self.column]

        if self.categorical:
            if col_value == self.value:
                return self.left.predict_one(x)
            else:
                return self.right.predict_one(x)
        else:
            if col_value < self.value:
                return self.left.predict_one(x)
            else:
                return self.right.predict_one(x)

In [47]:
class DecisionTree(object):
    '''
    A decision tree class.
    '''

    def __init__(self, impurity_criterion='entropy'):
        '''
        Initialize an empty DecisionTree.
        '''

        self.root = None  # root Node
        self.feature_names = None  # string names of features (for interpreting
                                   # the tree)
        self.categorical = None  # Boolean array of whether variable is
                                 # categorical (or continuous)
                                 # use in the _make_split method
        self.impurity_criterion = self._entropy \
                                  if impurity_criterion == 'entropy' \
                                  else self._gini

    def fit(self, X, y, feature_names=None):
        '''
        INPUT:
            - X: 2d numpy array
            - y: 1d numpy array
            - feature_names: numpy array of strings
        OUTPUT: None
        Build the decision tree.
        X is a 2 dimensional array with each column being a feature and each
        row a data point.
        y is a 1 dimensional array with each value being the corresponding
        label.
        feature_names is an optional list containing the names of each of the
        features.
        '''


        # This piece of code is used to provide feature names to the Decision tree
        if feature_names is None or len(feature_names) != X.shape[1]:
            # if the user has not provided feature names, just give them numbers
            self.feature_names = np.arange(X.shape[1])
        else:
            # otherwise, these are the names
            self.feature_names = feature_names

        # * Create True/False array of whether the variable is categorical
        # use a lambda function called is_categorical to determine if the variable is an instance
        # of str, bool or unicode - in that case is_categorical will be true
        # otherwise False. Look up the function isinstance()

        is_categorical = lambda x: isinstance(x, str) or \
                                   isinstance(x, bool) 
            
        # Each variable (organized by index) is given a label categorical or not
        self.categorical = np.vectorize(is_categorical)(X[0])

        # Call the build_tree function
        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y):
        '''
        INPUT:
            - X: 2d numpy array
            - y: 1d numpy array
        OUTPUT:
            - TreeNode
        Recursively build the decision tree. Return the root node.
        '''

        #  * initialize a root TreeNode
        node = TreeNode()
        # * set index, value, splits as the output of self._choose_split_index(X,y)
        index, value, splits = self._choose_split_index(X, y)

        # if no index is returned from the split index or we cannot split
        if index is None or len(np.unique(y)) == 1:
            # * set the node to be a leaf
            node.leaf = True
            # * set the classes attribute to the number of classes
            # * we have in this leaf with Counter()
            node.classes = Counter(y)
            # * set the name of the node to be the most common class in it
            node.name = node.classes.most_common(1)[0][0]

        else: # otherwise we can split (again this comes out of choose_split_index
            # * set X1, y1, X2, y2 to be the splits
            X1, y1, X2, y2 = splits
            # * the node column should be set to the index coming from split_index
            node.column = index
            # * the node name is the feature name as determined by
            #   the index (column name)
            node.name = self.feature_names[index]

            # * set the node value to be the value of the split
            node.value = value

            # * set the categorical flag of the node to be the category of the column
            node.categorical = self.categorical[index]

            # * now continue recursing down both branches of the split
            node.left = self._build_tree(X1, y1)
            node.right = self._build_tree(X2, y2)

        return node

    def _entropy(self, y):
        '''
        INPUT:
            - y: 1d numpy array
        OUTPUT:
            - float
        Return the entropy of the array y.
        '''

        total = 0
        # * for each unique class C in y
        for c in np.unique(y):
            # * count up the number of times the class C appears and divide by
            # * the total length of y. This is the p(C)
            # * add the entropy p(C) ln p(C) to the total
            p_C = np.sum(y == c) / float(len(y))
            total += p_C * np.log(p_C)
        return -total

    def _gini(self, y):
        '''
        INPUT:
            - y: 1d numpy array
        OUTPUT:
            - float
        Return the gini impurity of the array y.
        '''

        total = 0
        # * for each unique class C in y
        for c in np.unique(y):
            # * count up the number of times the class C appears and divide by
            # * the size of y. This is the p(C)
            # * add p(C)**2 to the total
            p_C = np.sum(y == c) / float(len(y))
            total += p_C**2
        return 1 - total

    def _make_split(self, X, y, split_index, split_value):
        '''
        INPUT:
            - X: 2d numpy array
            - y: 1d numpy array
            - split_index: int (index of feature)
            - split_value: int/float/bool/str (value of feature)
        OUTPUT:
            - X1: 2d numpy array (feature matrix for subset 1)
            - y1: 1d numpy array (labels for subset 1)
            - X2: 2d numpy array (feature matrix for subset 2)
            - y2: 1d numpy array (labels for subset 2)
        Return the two subsets of the dataset achieved by the given feature and
        value to split on.
        Call the method like this:
        X1, y1, X2, y2 = self._make_split(X, y, split_index, split_value)
        X1, y1 is a subset of the data.
        X2, y2 is the other subset of the data.
        '''

        # * slice the split column from X with the split_index
        split_column = X[:, split_index]
        # * if the variable of this column is categorical
        if self.categorical[split_index]:
            # * select the indices of the rows in the column
            #  with the split_value (T/F) into one set of indices (call them A)
            A = split_column == split_value
            # * select the indices of the rows in the column
            # that don't have the split_value into another
            #  set of indices (call them B)
            B = split_column != split_value
        # * else if the variable is not categorical
        else:
             # * select the indices of the rows in the column
            #  less than the split value into one set of indices (call them A)
            A = split_column < split_value
            # * select the indices of the rows in the column
            #  greater or equal to  the split value into
            # another set of indices (call them B)
            B = split_column >= split_value
            
        return X[A], y[A], X[B], y[B]

    def _information_gain(self, y, y1, y2):
        '''
        INPUT:
            - y: 1d numpy array
            - y1: 1d numpy array (labels for subset 1)
            - y2: 1d numpy array (labels for subset 2)
        OUTPUT:
            - float
        Return the information gain of making the given split.
        Use self.impurity_criterion(y) rather than calling _entropy or _gini
        directly.
        '''
        # * set total equal to the impurity_criterion
        total = self.impurity_criterion(y)
        
        e2 = len(y1)/len(y)*self.impurity_criterion(y1) + len(y2)/len(y)*self.impurity_criterion(y2)
        total -= e2
#         # * for each of the possible splits y1 and y2
#         for split in  
#             # * calculate the impurity_criterion of the split
#             imp_cri = self.impurity_criterion(split) 
#             # * subtract this value from the total, multiplied by split_size/y_size
#             total -= imp_cri * len(split)/
            
        return total

    def _choose_split_index(self, X, y):
        '''
        INPUT:
            - X: 2d numpy array
            - y: 1d numpy array
        OUTPUT:
            - index: int (index of feature)
            - value: int/float/bool/str (value of feature)
            - splits: (2d array, 1d array, 2d array, 1d array)
        Determine which feature and value to split on. Return the index and
        value of the optimal split along with the split of the dataset.
        Return None, None, None if there is no split which improves information
        gain.
        Call the method like this:
        index, value, splits = self._choose_split_index(X, y)
        X1, y1, X2, y2 = splits
        '''

        # set these initial variables to None
        split_index, split_value, split = None, None, None
        # we need to keep track of the maximum entropic gain
        max_gain = 0

        # * for each column in X
        for col in range(X.shape[1]):
            # * set an array called values to be the
            # unique values in that column (use np.unique)
            values = np.unique(X[:, col])
            # if there are less than 2 values, move on to the next column
            if len(values) < 2:
                continue

            # * for each value V in the values array
            for val in values:
                # * make a temporary split (using the column index and V) with make_split
                temporary_split = self._make_split(X, y, col, val)
                # * calculate the information gain between the original y, y1 and y2
                X1, y1, X2, y2 = temporary_split
                gain = self._information_gain(y, y1, y2)
                # * if this gain is greater than the max_gain
                if gain > max_gain:

                    # * set max_gain, split_index, and split_value to be equal
                    # to the current max_gain, column and value
                    # * set the output splits to the current split setup (X1, y1, X2, y2)
                    split = temporary_split
                    max_gain, split_index, split_value = gain, col, val
                   
        return split_index, split_value, split 
    
    
    def predict(self, X):
        '''
        INPUT:
            - X: 2d numpy array
        OUTPUT:
            - y: 1d numpy array
        Return an array of predictions for the feature matrix X.
        '''

        return np.apply_along_axis(self.root.predict_one, axis=1, arr=X)

    def __str__(self):
        '''
        Return string representation of the Decision Tree. This will allow you to $:print tree
        '''
        return str(self.root)

In [49]:
class RandomForest(object):
    '''A Random Forest class'''

    def __init__(self, num_trees, num_features):
        '''
           num_trees:  number of trees to create in the forest:
        num_features:  the number of features to consider when choosing the
                           best split for each node of the decision trees
        '''
        self.num_trees = num_trees
        self.num_features = num_features
        self.forest = None

    def fit(self, X, y):
        '''
        X:  two dimensional numpy array representing feature matrix
                for test data
        y:  numpy array representing labels for test data
        '''
        self.forest = self.build_forest(X, y, self.num_trees, X.shape[0], \
                                        self.num_features)

    def build_forest(self, X, y, num_trees, num_samples, num_features):

        # * Return a list of num_trees DecisionTrees.
        row, col = X.shape
        forest = []
        
        for tree in range(num_trees):
            # create a random set of X_samples with replacement
            X_samp = np.random.randint(row, size=num_samples)
            # create a random permutation of features (list)[sliced to length num_features]
            y_samp = np.random.permutation(col)[:num_features]
            X_tree = X[X_samp,:][:,y_samp]
            y_tree = y[X_samp]
            tree = DecisionTree()
            tree.fit(X_tree, y_tree, feature_names=y_samp)
            forest.append(tree)
            
        return forest

    def predict(self, X):

        '''
        Return a numpy array of the labels predicted for the given test data.
        '''
       
        # * Each one of the trees is allowed to predict on the same row of input data. The majority vote
        # is the output of the whole forest. This becomes a single prediction.
        
        predictions  = []
        for tree in self.forest:
            predict = tree.predict(X[:,tree.feature_names])
            predictions.append(predict)
        # thank you Tristan! Count along the columns and find the most common    
        return np.array(np.apply_along_axis(lambda col: Counter(col).most_common()[0][0], arr=predictions, axis=0))

    
    def score(self, X, y):

        '''
        Return the accuracy of the Random Forest for the given test data.
        '''

        # * In this case you simply compute the accuracy formula as we have defined in class. Compare predicted y to
        # the actual input y.
        
        return sum(self.predict(X) == y) / len(y)